# Filtering polarimertric cycles for SPHERE-DC reduced data without normalisation based on the peakI and copying into separate folder.
## Mean combining for the I


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import fnmatch
import os
from astropy.io import fits
from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse
from scipy import interpolate
import cv2
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
from textwrap import wrap
import scipy.ndimage as ndimage
from matplotlib.gridspec import GridSpec
from matplotlib import colors
import functions as f
import pandas as pd
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

#calculating mean-combined versions and multiply

def load_and_make_polar(dirdat,camera):
    dir = dirdat
    qfile = '*sci'+camera+'.fits'
    files = os.listdir(dir)
    for file in files:
        if fnmatch.fnmatch(file, qfile):
            hdulq = fits.open(dir + file)
            iq = hdulq[0].data[0]
            q = hdulq[0].data[1]
            iu = hdulq[0].data[2]
            u = hdulq[0].data[3]
            n=q.shape[0]
             #Creating grid         
            xr = np.linspace(-n/2, n/2, num=n)
            yr = np.linspace(-n/2, n/2, num=n)
            x0 = 0.5
            y0 = 0.5
            xr = xr-x0
            yr = yr-y0
            Xr, Yr = np.meshgrid(xr, yr)
            R = np.sqrt(Xr**2 + Yr**2)
            phi = np.arctan(Yr/Xr)
            i=(iq+iu)/2
            q_phi=q*np.cos(2*phi)+u*np.sin(2*phi)
            q_phi = (q_phi > 0)*q_phi +  (q_phi <=0 )*1e-10
            u_phi=q*np.sin(2*phi)+u*np.cos(2*phi)
            pi=np.sqrt(q*q+u*u)
            aolp=0.5*np.arctan2(u, q)+np.pi/2
    return iq,q,iu,u,i,q_phi,u_phi,pi,aolp,R,phi



def load_and_make_polar_umon(dirdat,cycle,camera):
    #q
    dir = dirdat+'q_corr/'
    qfile = '*_'+cycle+'_'+camera+'.fits'
    files = os.listdir(dir)
    for file in files:
        if fnmatch.fnmatch(file, qfile):
            hdulq = fits.open(dir + file)
            q = hdulq[0].data
            n=q.shape[0]
    #u
    dir = dirdat+'u_corr/'
    qfile = '*_'+cycle+'_'+camera+'.fits'
    files = os.listdir(dir)
    for file in files:
        if fnmatch.fnmatch(file, qfile):
            hdulq = fits.open(dir + file)
            u = hdulq[0].data
    #qphi
    dir = dirdat+'qphi_corr/'
    qfile = '*_'+cycle+'_'+camera+'.fits'
    files = os.listdir(dir)
    for file in files:
        if fnmatch.fnmatch(file, qfile):
            hdulq = fits.open(dir + file)
            qphi = hdulq[0].data
            
    #uphi
    dir = dirdat+'uphi_corr/'
    qfile = '*_'+cycle+'_'+camera+'.fits'
    files = os.listdir(dir)
    for file in files:
        if fnmatch.fnmatch(file, qfile):
            hdulq = fits.open(dir + file)
            uphi = hdulq[0].data
            
            
            
    #iq,iu
    dir = dirdat+'sci/'
    qfile = '*_'+cycle+'_'+camera+'.fits'
    files = os.listdir(dir)
    for file in files:
        if fnmatch.fnmatch(file, qfile):
            hdulq = fits.open(dir + file)
            iq = hdulq[0].data[0]
            iu = hdulq[0].data[2]
            
            
            
             #Creating grid         
            xr = np.linspace(-n/2, n/2, num=n)
            yr = np.linspace(-n/2, n/2, num=n)
            x0 = 0.5
            y0 = 0.5
            xr = xr-x0
            yr = yr-y0
            Xr, Yr = np.meshgrid(xr, yr)
            R = np.sqrt(Xr**2 + Yr**2)
            phi = np.arctan(Yr/Xr)
            
            i=(iq+iu)/2
            
            qphi = (qphi > 0)*qphi +  (qphi <=0 )*1e-10
            pi=np.sqrt(q*q+u*u)
            aolp=0.5*np.arctan2(u, q)+np.pi/2
            
    return iq,q,iu,u,i,qphi,uphi,pi,aolp,R,phi





def Loadimage(dirdat,filename):
    dir =dirdat
    psfile =  filename
    files = os.listdir(dir)
    for file in files:
        if fnmatch.fnmatch(file, psfile):
            hdulPSF = fits.open(dir + file)
            fit = hdulPSF[0].data

            
    return fit



In [2]:
def load_tel_corr(dirdat,camera,process):
    dir = dirdat
    files = os.listdir(dir)
    qfile = 'qcorr_'+str(process)+'_'+str(camera)+'.fits'
    ufile = 'ucorr_'+str(process)+'_'+str(camera)+'.fits'
    qphifile = 'qphi_'+str(process)+'_'+str(camera)+'.fits'
    uphifile = 'uphi_'+str(process)+'_'+str(camera)+'.fits'
    
    for file in files:
        if fnmatch.fnmatch(file, qfile):
            hdulq = fits.open(dir + file)
            q = hdulq[0].data
        if fnmatch.fnmatch(file, ufile):
            hdulq = fits.open(dir + file)
            u = hdulq[0].data
        #if fnmatch.fnmatch(file, qphifile):
        #    hdulq = fits.open(dir + file)
        #    q_phi = hdulq[0].data
        #if fnmatch.fnmatch(file, uphifile):
        #    hdulq = fits.open(dir + file)
        #    u_phi = hdulq[0].data    
            
            
    n=q.shape[0]
     #Creating grid         
    xr = np.linspace(-n/2, n/2, num=n)
    yr = np.linspace(-n/2, n/2, num=n)
    x0 = 0.5
    y0 = 0.5
    xr = xr-x0
    yr = yr-y0
    Xr, Yr = np.meshgrid(xr, yr)
    R = np.sqrt(Xr**2 + Yr**2)
    phi = np.arctan(Yr/Xr)
    q_phi=q*np.cos(2*phi)+u*np.sin(2*phi)
    q_phi = (q_phi > 0)*q_phi +  (q_phi <=0 )*1e-10
    u_phi=q*np.sin(2*phi)+u*np.cos(2*phi)    
    
    return q,u,q_phi,u_phi


def createfolder(dirName):
    try:
    # Create target Directory
       os.mkdir(dirName)
    except FileExistsError:
        print()#("Directory " , dirName ,  " already exists")   
        
        
def maskcrit(aolp,R):
    phi = (aolp-90)  
    n=aolp.shape[0]
    critarray=np.zeros_like(phi)
    for ix in range (0,n):
        for iy in range(0,n):
            if phi[ix,iy]>180:
                phi[ix,iy]=phi[ix,iy]-180
            
    for ix in range (2,n-2):
        for iy in range(2,n-2):
            if R[ix,iy]>=1:            
                datapix=[]
                for (iix,iiy) in [(ix,iy),(ix-1,iy),(ix+1,iy),(ix,iy-1),(ix,iy+1)]: 
                    if R[iix,iiy]>=1:
                        datapix.append(abs(phi[iix,iiy]))
                            
                
                crit=np.std(datapix)               
                critarray[ix,iy]=crit
                
    for ix in range (0,n-1):
        for iy in range(0,n-1):
            if critarray[ix,iy]==0:
                critarray[ix,iy]=np.max(critarray)
    medianstd=np.nanmedian(critarray)
    
    mask=(critarray<=medianstd)#*R([ix,iy]>=1)
    return mask
        
def plot_AoLP(ps,R,Q,U,I,Q_PHI,PI,AOLP,title,save):
    # First, we plot the background image
    fig = plt.figure(figsize=(8,8))
    i_plot = fig.add_subplot(111)
    n = I.shape[0]
    d = n * ps / 2

    im1=i_plot.imshow(np.arcsinh(Q_PHI), origin='lower',extent=(-d, d, d, -d))

    fig.colorbar(im1, orientation='vertical')

    plt.xlabel('mas')
    plt.ylabel("mas")
    plt.tight_layout(pad=0.1)     

    # ranges of the axis
    xx0, xx1 = i_plot.get_xlim()
    yy0, yy1 = i_plot.get_ylim()

    # binning factor
    factor = [4, 4]

    # re-binned number of points in each axis
    nx_new = PI.shape[1] // factor[0]
    ny_new = PI.shape[0] // factor[1]

    # These are the positions of the quivers
    X,Y = np.meshgrid(np.linspace(xx0,xx1,nx_new,endpoint=True),
                      np.linspace(yy0,yy1,ny_new,endpoint=True))
    # bin the data
    I_bin = I.reshape(nx_new, factor[0], ny_new, factor[1]).sum(3).sum(1)
    Q_bin = Q.reshape(nx_new, factor[0], ny_new, factor[1]).sum(3).sum(1)
    U_bin = U.reshape(nx_new, factor[0], ny_new, factor[1]).sum(3).sum(1)
    Q_phi_bin = Q_PHI.reshape(nx_new, factor[0], ny_new, factor[1]).sum(3).sum(1)
    PI_bin=PI.reshape(nx_new, factor[0], ny_new, factor[1]).sum(3).sum(1)
    R_bin=R.reshape(nx_new, factor[0], ny_new, factor[1]).sum(3).sum(1)
    aolp_bin=AOLP.reshape(nx_new, factor[0], ny_new, factor[1]).sum(3).sum(1)

    # polarization angle
    psi=0.5*np.arctan2(U_bin, Q_bin)

    #psi=aolp_bin
    #i_plot.imshow(psi, origin='lower',extent=(-d, d, d, -d))

    # polarization fraction
    frac =Q_phi_bin/I_bin
    #frac=dolp_v[adc:bdc,adc:bdc]
    # mask to show only alighned

    mask1=maskcrit(psi,R_bin)
    mask2=Q_phi_bin>=10
    mask=mask2#*mask1
    #print('max DoLP in region %.3f percent'%(np.max(frac[mask])*100))

    #+pi/2 because quiver function start counting from the horizontal axis counterclockwise 
    #and the AoLP have to start from North to East (which is also counterclockvise)
    pixX = frac*np.cos(psi+np.pi/2) # X-vector 
    pixY = frac*np.sin(psi+np.pi/2) # Y-vector

    # keyword arguments for quiverplots
    quiveropts = dict(headlength=0, headwidth=1, pivot='middle', color='w')
    i_plot.quiver(X[mask], Y[mask], pixX[mask], pixY[mask],scale=2, **quiveropts)
    plt.title(title)
    plt.savefig(save,bbox_inches='tight', pad_inches=0.1)
    #plt.show()#(dirName+"aolp.jpeg",bbox_inches='tight', pad_inches=0.1)
    plt.close()
    
def gaus(x,a,x0,sigma):
            return a*np.exp(-(x-x0)**2/(2*sigma**2))

    
def find_FWHM (PSF,n,ps,figfolder,title):             #resolution
    middle=int(n/2)

    y1=PSF[middle,middle-60:middle+60]
    y2=PSF[middle-60:middle+60,middle]

    xdata = np.linspace(middle-60,middle+60, num=len(y1))


    n_gauss = len(xdata) #the number of data
    amp=np.max(y1)
    mean = np.sum(xdata * y1) / sum(y1)
    sigma = np.sqrt(sum(y1 * (xdata - mean)**2) / sum(y1))

    popt1,pcov1 = curve_fit(gaus,xdata,y1,p0=[amp,mean,sigma])
    popt2,pcov2 = curve_fit(gaus,xdata,y2,p0=[amp,mean,sigma])


    plt.plot(xdata, y1, 'o', label='vertical')
    plt.plot(xdata, gaus(xdata,*popt1), '-', label='fit1')
    plt.plot(xdata, y2, 'o', label='horizotal')
    plt.plot(xdata, gaus(xdata,*popt2), '-', label='fit2')
    plt.legend()
    plt.title(title)
    plt.savefig(figfolder+title+'.jpeg',bbox_inches='tight', pad_inches=0.1)
    plt.close()

    fwhm1=2*np.sqrt(2*math.log(2))*popt1[2]
    fwhm2=2.355*popt2[2]


    fwhm=(abs(fwhm1)+abs(fwhm2))/2

    return fwhm

def ap_fixed_in(rad,R,q,u,PSF):            
    mask = (R <= rad)
    q_sum = np.sum(q[mask])
    u_sum = np.sum(u[mask])
    psf_sum = np.sum(PSF[mask])
    q_i_percent = q_sum / psf_sum * 100
    u_i_percent = u_sum / psf_sum * 100
    return psf_sum, q_sum, u_sum, q_i_percent, u_i_percent

def ap(radin,radout,R,q,u,PSF):
    mask = (R <= radout) * (R >= radin)
    q_sum = np.sum(q[mask])
    u_sum = np.sum(u[mask])
    psf_sum = np.sum(PSF[mask])
    q_i_percent = q_sum / psf_sum * 100
    u_i_percent = u_sum / psf_sum * 100

    return psf_sum, q_sum, u_sum, q_i_percent, u_i_percent

def Loadimagespsf_AR(star):
    dir = '/media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/classic_reduction/'+star+'/'
 
    psfile =  '*REDUCED_I.fits'
    files = os.listdir(dir)
    for file in files:
        if fnmatch.fnmatch(file, psfile):
            hdulPSF = fits.open(dir + file)
            PSFv = hdulPSF[0].data[0]
            PSFi = hdulPSF[0].data[1]

            
    return PSFv,PSFi


def unres_correction(iq,iu,q,u):

    n=q.shape[0]
     #Creating grid         
    xr = np.linspace(-n/2, n/2, num=n)
    yr = np.linspace(-n/2, n/2, num=n)
    x0 = 0.5
    y0 = 0.5
    xr = xr-x0
    yr = yr-y0
    Xr, Yr = np.meshgrid(xr, yr)
    R = np.sqrt(Xr**2 + Yr**2)
    phi = np.arctan(Yr/Xr)
    i=(iq+iu)/2
    mask=(R<=3)

    q_over_i=np.divide(q,i,where=i!=0)   
    cq=np.median(q_over_i[mask]) #for median normal as in IRDIS
    u_over_i=np.divide(u,i,where=i!=0)    
    cu=np.median(u_over_i[mask]) #for median normal as in IRDIS
    q_corr=q-cq*iq
    u_corr=u-cu*iu

    q_phi=q_corr*np.cos(2*phi)+u_corr*np.sin(2*phi)
    q_phi= (q_phi> 0)*q_phi +  (q_phi <=0 )*1e-10
    u_phi=q_corr*np.sin(2*phi)+u_corr*np.cos(2*phi)
    pi=np.sqrt(q_corr*q_corr+u_corr*u_corr)
    aolp=0.5*np.arctan2(u, q)+np.pi/2

    return iq,q_corr,iu,u_corr,i,q_phi,u_phi,pi,aolp,R,phi



def load_I(dirdat,camera):
    dir = dirdat
    qfile = '*sci'+camera+'.fits'
    files = os.listdir(dir)
    for file in files:
        if fnmatch.fnmatch(file, qfile):
            hdulq = fits.open(dir + file)
            iq = hdulq[0].data[0]
            q = hdulq[0].data[1]
            iu = hdulq[0].data[2]
            u = hdulq[0].data[3]
            n=q.shape[0]
            i=(iq+iu)/2
            
    return iq,iu,i





## Downloding files and selecting top percent by peak I

In [3]:

#stars=['HD83878','IRAS08544-4431_dc_notnorm','HD75885', 'AR_Pup_dc_notnorm','UMon','HR4049_20190108','HR4049_20190107','HD71253','HD94680','HD96314','HD98025','V709_Car','HR4226']
stars=['HD75885', 'AR_Pup_dc_notnorm']
star=stars[1]

starnames = {'HD75885':'HD75885','AR_Pup':'AR_Pup','HR4049/2019-01-08':'HR4049-2019-01-08','HR4049/2019-01-07':'HR4049-2019-01-07','IRAS08544-4431':'IRAS08544-4431','UMon':'UMon','AR_Pup_flat4':'AR_Pup_flat4','V709_Car':'V709_Car','UMon_calibV390':'UMon_calibV390','HR4224':'HR4224','UMon_Katya':'UMon_Katya'}
fittypes=['1', '2']

band=['V','I']
ps=3.6


if star=='AR_Pup':
    dirdat = '//media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/individual_cycles/AR_Pup/Normalised/'  #For AR_Pup
    input_filename =dirdat+'AR_Pup_indiv_process_list.csv'
elif star=='AR_Pup_dc_notnorm':
    dirdat = '//media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/individual_cycles/AR_Pup/Not_normalised/'  #For AR_Pup
    input_filename =dirdat+'AR_Pup_no_norm_indiv_process_list.csv'

elif star=='IRAS08544-4431':
    dirdat = '//media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/individual_cycles/IRAS08544-4431/Normalised/'  #For IRAS08
    input_filename =dirdat+'IRAS08process_list.csv'
elif star=='IRAS08544-4431_dc_notnorm':
    dirdat = '//media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/individual_cycles/IRAS08544-4431/Not_normalised/'  #For IRAS08
    input_filename =dirdat+'IRAS08_not_norm_indiv_process_list.csv'

elif star=='UMon':
    dirdat = '//media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/UMon_data/HansMartin/'
    input_filename =dirdat+'UMon_cycle_list.csv'

else:
    dirdat = '//media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/individual_cycles/'+star+'/' 
    input_filename =dirdat+'process_list.csv'
    
table = pd.read_csv(input_filename)
table['reference']= table['reference'].str.replace(' ', '_') #ix in case there are spaces in the name


figfolder='//media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/'+'Quality_indiv_cycles/'+ star+'/'


output_filename = figfolder+star+'_V_Original_AP_per_cycle.txt'
df_orig_v = pd.read_csv(output_filename, delim_whitespace=True)
output_filename = figfolder+star+'_I_Original_AP_per_cycle.txt'
df_orig_i = pd.read_csv(output_filename, delim_whitespace=True)

#stars = ['HD75885','AR_Pup','HR4049/2019-01-08','HR4049/2019-01-07','IRAS08544-4431','UMon','V709_Car','UMon_calibV390','HR4224']


processes=table['id']

reference={process:ref for process,ref in zip(table['id'], table['reference'])}
process_dict={ref:process for process,ref in zip(table['id'], table['reference'])}

percentile_25 = np.percentile(df_orig_v['Peak_i(Strehl)'], 25)
print(percentile_25)
print(np.max(df_orig_v['Peak_i(Strehl)']))

#filtering list of processes
filtered_process_dict = {key: value for key, value in process_dict.items() if df_orig_v[df_orig_v['reference'] == key]['Peak_i(Strehl)'].values[0] >=percentile_25}

print(filtered_process_dict)
print(len(filtered_process_dict))

#for key, value in process_dict.items():
 #   print(key, df_orig_v[df_orig_v['reference'] == key]['FWHM'].values[0])

0.000532
0.000752
{'AR_Pup_17c_00_12-13_not_norm': 481358, 'AR_Pup_16c_00_10-11_not_norm': 481357, 'AR_Pup_15c_00_08-09_not_norm': 481356, 'AR_Pup_14c_00_06-07_not_norm': 481355, 'AR_Pup_13c_00_04-05_not_norm': 481354, 'AR_Pup_12c_00_02-03_not_norm': 481353, 'AR_Pup_11c_00_00-02(1)_not_norm': 481352, 'AR_Pup_10c_23_58-00(1)_not_norm': 481351, 'AR_Pup_9c_23_56-58(1)_not_norm': 481350, 'AR_Pup_7c_23_53-54(1)_not_norm': 481348, 'AR_Pup_6c_23_51-52_not_norm': 481347, 'AR_Pup_5c_23_49-50_not_norm': 481346, 'AR_Pup_4c_23_47-48_not_norm': 481345, 'AR_Pup_3c_23_45-46_not_norm': 481344, 'AR_Pup_2c_23_43-44_not_norm': 481343}
15


In [4]:
import shutil
import csv

fitsfolder='//media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/'+'filtered/'
createfolder(fitsfolder)
fitsfolder=fitsfolder+star+'/'
createfolder(fitsfolder)
fitsfolder=fitsfolder+'selected/'
createfolder(fitsfolder)



with open(fitsfolder+'filtered_process_list.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['reference', 'process'])
    
    for reference,process in filtered_process_dict.items():
        csv_writer.writerow([reference,process])
        
        
        
for fittype in fittypes:
    
    fitsfolder0=fitsfolder+'cam'+fittype+'/'
    createfolder(fitsfolder0)
    print(band[int(fittype)-1])
    for key in filtered_process_dict.keys():
        process=filtered_process_dict[key]
        if star!='UMon':
            dirName=dirdat+'SPHERE_DC_DATA/'+str(process)+'/'
            qfile = '*sci'+fittype+'.fits'
            files = os.listdir(dirName)
            for file in files:
                if fnmatch.fnmatch(file, qfile):
                    shutil.copyfile(dirName+file,fitsfolder0+str(process)+'_'+file)
                
                
                
                
        if star=='UMon':
            for folder in ['q_corr','u_corr','qphi_corr','uphi_corr','sci']:
                fitsfolder_sub=fitsfolder0+folder+'/'
                createfolder(fitsfolder_sub)    
                dirName=dirdat+folder+'/'
                qfile = '*'+process+'_'+fittype+'.fits'

                files = os.listdir(dirName)
                for file in files:
                    if fnmatch.fnmatch(file, qfile):
                        shutil.copyfile(dirName+file,fitsfolder_sub+str(process)+'_'+file)
                       
                
                
                
        





V

I


## Mean combining I total and saving as fits

In [5]:
#Images of the mean_combined data
figfolder='//media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/'+'filtered/'
createfolder(figfolder)
figfolder=figfolder+star+'/'
createfolder(figfolder)
print("{:<30} {:<10}  \n".format('reference', 'Peak_I(Strehl)'))



for fittype in fittypes:
    print(band[int(fittype)-1])
    iq_arr=[]
    iu_arr=[]
    q_arr=[]
    u_arr=[]
    i_arr=[]
    q_phi_arr=[]
    u_phi_arr=[]
    pi_arr=[]
    
    for key in filtered_process_dict.keys():
        process=filtered_process_dict[key]
        if star=='UMon':
            dirName=dirdat
        else:
            dirName=dirdat+'SPHERE_DC_DATA/'+str(process)+'/'
        if star=='UMon':
            iq,_,iu,_,i,_,_,_,_,_,_= load_and_make_polar_umon(dirName,process,fittype)
        else:        
            iq,iu,i= load_I(dirName,fittype)
        iq_arr.append(iq)
        iu_arr.append(iu)
        i_arr.append(i)
        
    iq_arr=np.array(iq_arr)
    iu_arr=np.array(iu_arr)
    i_arr=np.array(i_arr)
    
    
    iq_mean_comb=np.mean(iq_arr,axis=0)
    iu_mean_comb=np.mean(iu_arr,axis=0)
    i_mean_comb=np.mean(i_arr,axis=0)
    
        
    n=iq_mean_comb.shape[0]
    if star=='AR_Pup' or star=='AR_Pup_dc_notnorm':
        lim=100       
    else:
        lim=50       
            
    xr = np.linspace(-n/2, n/2, num=n)
    yr = np.linspace(-n/2, n/2, num=n)
    x0 = 0.5
    y0 = 0.5
    xr = xr-x0
    yr = yr-y0
    Xr, Yr = np.meshgrid(xr, yr)
    R = np.sqrt(Xr**2 + Yr**2)
    
    mask=(R<=1500/ps)
    I_sum=np.sum(i_mean_comb[mask])
    
    fwhm=find_FWHM (i_mean_comb,n,ps,figfolder,band[int(fittype)-1]+'_gauss_mean_combined')
    
    
    mask=(R<=1500/ps)
    #calculating what is the ratio of peak brightness to the total. For I it refers to the observational conditions and is a proxy of strehl
    normalisation=np.sum(i_mean_comb[mask])
    peak_i=np.max(i_mean_comb*(R<200))/normalisation
    
    line = "{:<30} {:<10.3f} \n".format('raw',peak_i)
    print(line)
        
    
    
    filename={0:'I'}

    for fileindex,image1 in enumerate([i_mean_comb]):
        out_fits = fits.HDUList(fits.PrimaryHDU(image1))                  # create output fits structure
        out_fits.writeto(figfolder+star+'_'+band[int(fittype)-1]+'_'+filename[fileindex]+'_meancombined.fits', overwrite = True)                       # write output

        image_an = image1*(R<250)
        image=np.arcsinh(image_an)
        fig, ax = plt.subplots()
        f.plottingroutinemas(image,lim,ps,n,star,ax)        
        f.scale_mas(star,ax)  

        plt.title(star+', '+filename[fileindex]+', '+band[int(fittype)-1]+', mean combined') 
        plt.savefig(figfolder+star+'_'+band[int(fittype)-1]+'_'+filename[fileindex]+'_'+'_meancombined.jpeg',bbox_inches='tight', pad_inches=0.1)
        #plt.show()
        plt.close()



        
        
        
        




reference                      Peak_I(Strehl)  

V
raw                            0.001      

I
raw                            0.001      

